In [58]:
# Dependencies
import requests
import json
import prettytable
import pandas as pd

In [59]:
# Series IDs are for the categories representative of the metrics used by US Bureau of Labor
# JTU000000480000000JOL: Job openings in the state of Texas in all non-farm industries, with companies of all sizes,
# Values shown in thousands

#LAUST480000000000006: Labor Force in state of Texas
#LAUST480000000000005: Employment # in state of Texas
#LAUST480000000000004: Unemployment in # in the state of Texas
#LAUST480000000000003: Unemployment Rate in the state in Texas 


In [60]:
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['JTU000000480000000JOL','LAUST480000000000006', 'LAUST480000000000005', 'LAUST480000000000003', 'LAUST480000000000004'],"startyear":"2018", "endyear":"2022"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()
json_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 343,
 'message': [],
 'Results': {'series': [{'seriesID': 'JTU000000480000000JOL',
    'data': [{'year': '2022',
      'period': 'M12',
      'periodName': 'December',
      'latest': 'true',
      'value': '855',
      'footnotes': [{'code': 'P', 'text': 'preliminary'}]},
     {'year': '2022',
      'period': 'M11',
      'periodName': 'November',
      'value': '811',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M10',
      'periodName': 'October',
      'value': '962',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M09',
      'periodName': 'September',
      'value': '1028',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M08',
      'periodName': 'August',
      'value': '971',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M07',
      'periodName': 'July',
      'value': '1115',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M06',
      'peri

In [61]:
# Used this to test JSON traversal
# There were 885,000 non-farm job openings in Texas in Dec 2022
json_data["Results"]["series"][0]["data"][0]["value"]
len(json_data["Results"]["series"][0]["data"])

60

In [62]:
# Used this to test JSON traversal
# Get the year
json_data["Results"]["series"][0]["data"][0]["year"]

'2022'

In [64]:
# Used this to test JSON traversal
# There December 2022, there were 14M people in the labor force
json_data["Results"]["series"][1]["data"][0]["value"]
json_data["Results"]["series"][1]["data"]

[{'year': '2022',
  'period': 'M12',
  'periodName': 'December',
  'value': '14773939',
  'footnotes': [{'code': 'R',
    'text': 'Data were subject to revision on March 1, 2023.'}]},
 {'year': '2022',
  'period': 'M11',
  'periodName': 'November',
  'value': '14765864',
  'footnotes': [{'code': 'R',
    'text': 'Data were subject to revision on March 1, 2023.'}]},
 {'year': '2022',
  'period': 'M10',
  'periodName': 'October',
  'value': '14731841',
  'footnotes': [{'code': 'R',
    'text': 'Data were subject to revision on March 1, 2023.'}]},
 {'year': '2022',
  'period': 'M09',
  'periodName': 'September',
  'value': '14685013',
  'footnotes': [{'code': 'R',
    'text': 'Data were subject to revision on March 1, 2023.'}]},
 {'year': '2022',
  'period': 'M08',
  'periodName': 'August',
  'value': '14674216',
  'footnotes': [{'code': 'R',
    'text': 'Data were subject to revision on March 1, 2023.'}]},
 {'year': '2022',
  'period': 'M07',
  'periodName': 'July',
  'value': '14678367'

In [65]:
# Store the year, period, and periodName, and # of job openings in columns of the dataframe
year = []
period = []
periodName = []
value = []
num_series = len(json_data["Results"]["series"][0]["data"])
series_data = json_data["Results"]["series"][0]["data"]

for i in range(num_series):
    year.append(series_data[i]["year"])
    period.append(series_data[i]["period"])
    periodName.append(series_data[i]["periodName"])
    value.append(series_data[i]["value"])

# Create a dataframe from these columns
job_openings = pd.DataFrame({"Year": year,
                            "Period": period,
                            "Month": periodName,
                            "# of job openings (thousands)": value})

# Since the last column contains string values, we convert them to integers
job_openings["# of job openings (thousands)"] = job_openings["# of job openings (thousands)"].astype(int)

# Show # of job openings in the actual thousand values
job_openings["# of job openings"] = job_openings["# of job openings (thousands)"]*1000

# Drop the duplicate column
job_openings = job_openings.drop(columns=["# of job openings (thousands)"])

# Print the dataframe
# job_openings

In [66]:
# Store the year, period, and periodName, and labor force # in columns of the dataframe
year1 = []
period1 = []
periodName1 = []
value1 = []
num_series1 = len(json_data["Results"]["series"][1]["data"])
series_data1 = json_data["Results"]["series"][1]["data"]

for i in range(num_series1):
    year1.append(series_data1[i]["year"])
    period1.append(series_data1[i]["period"])
    periodName1.append(series_data1[i]["periodName"])
    value1.append(series_data1[i]["value"])

# Create a dataframe from these columns
labor_force = pd.DataFrame({"Year": year1,
                            "Period": period1,
                            "Month": periodName1,
                            "Labor Force": value1})

# Print the dataframe
# labor_force

In [67]:
# Store the year, period, and periodName, and the employment # in columns of the dataframe
year2 = []
period2 = []
periodName2 = []
value2 = []
num_series2 = len(json_data["Results"]["series"][2]["data"])
series_data2 = json_data["Results"]["series"][2]["data"]

for i in range(num_series2):
    year2.append(series_data2[i]["year"])
    period2.append(series_data2[i]["period"])
    periodName2.append(series_data2[i]["periodName"])
    value2.append(series_data2[i]["value"])

# Create a dataframe from these columns
employment_number = pd.DataFrame({"Year": year2,
                            "Period": period2,
                            "Month": periodName2,
                            "Employment Number": value2})

# Print the dataframe
# employment_number

In [70]:
# Store the year, period, and periodName, and the unemployment rate in columns of the dataframe
year3 = []
period3 = []
periodName3 = []
value3 = []
num_series3 = len(json_data["Results"]["series"][3]["data"])
series_data3 = json_data["Results"]["series"][3]["data"]

for i in range(num_series3):
    year3.append(series_data3[i]["year"])
    period3.append(series_data3[i]["period"])
    periodName3.append(series_data3[i]["periodName"])
    value3.append(series_data3[i]["value"])

# Create a dataframe from these columns
unemployment_rate = pd.DataFrame({"Year": year3,
                            "Period": period3,
                            "Month": periodName3,
                            "Unemployment Rate": value3})

# Print the dataframe
# unemployment_rate

In [71]:
# Store the year, period, and periodName, and the unemployment # in columns of the dataframe
year4 = []
period4 = []
periodName4 = []
value4 = []
num_series4 = len(json_data["Results"]["series"][4]["data"])
series_data4 = json_data["Results"]["series"][4]["data"]

for i in range(num_series4):
    year4.append(series_data4[i]["year"])
    period4.append(series_data4[i]["period"])
    periodName4.append(series_data4[i]["periodName"])
    value4.append(series_data4[i]["value"])

# Create a dataframe from these columns
unemployment_number = pd.DataFrame({"Year": year4,
                            "Period": period4,
                            "Month": periodName4,
                            "Unemployment Number": value4})


# Print the dataframe
# unemployment_number

In [75]:
# Merge dataframes together
merge_1 = pd.merge(job_openings, labor_force, how = "outer", on=["Year", "Period", "Month"])
merge_2 = pd.merge(merge_1, employment_number, how = "outer", on=["Year", "Period", "Month"])
merge_3 = pd.merge(merge_2, unemployment_rate, how = "outer", on=["Year", "Period", "Month"])
merge_4 = pd.merge(merge_3, unemployment_number, how = "outer", on=["Year", "Period", "Month"])
merge_4


,Year,Period,Month,# of job openings,Labor Force,Employment Number,Unemployment Rate,Unemployment Number
0,2022,M12,December,855000,14773939,14260768,3.5,513171
1,2022,M11,November,811000,14765864,14222585,3.7,543279
2,2022,M10,October,962000,14731841,14189171,3.7,542670
3,2022,M09,September,1028000,14685013,14142120,3.7,542893
4,2022,M08,August,971000,14674216,14094855,3.9,579361
5,2022,M07,July,1115000,14678367,14088346,4.0,590021
6,2022,M06,June,1038000,14673500,14058566,4.2,614934
7,2022,M05,May,967000,14616414,14085102,3.6,531312
8,2022,M04,April,1104000,14592934,14076764,3.5,516170
9,2022,M03,March,999000,14633464,14080750,3.8,552714
